# DP 4 Drone Race
by Emily Lory and Conan Zhang

In [1]:
# Import the quadrotor project. This module is used to simulate, render, and plot a quadrotor dynamic system
from sympy import * 
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import ae353_quadrotor
init_printing()

In [2]:
# Get the gravity in m/s^2
from ae353_quadrotor import get_gravity
planet = "Earth"
grav = get_gravity(planet, verbose=True)

In [3]:
# Mass and moment of inertia of the bus
mass = 0.5    # The mass of the quadrotor in kg
Ixx = 0.0023  # The moment of inertia of the quadrotor in 
Iyy = 0.0023  # The mass of the quadrotor in kg
Izz = 0.0040  # The mass of the quadrotor in kg
lxy = 0.25    # The x or y distance from the quadrotor frame to the mocap markers in meters
lz = 0.046875 # The z distance from the quadrotor frame to the mocap markers in meters

In [4]:
# Define position
px_inW, py_inW, pz_inW = symbols('p_x, p_y, p_z')

# Define velocity
vx_inB, vy_inB, vz_inB = symbols('v_x, v_y, v_z')

# Define roll (phi), pitch (theta), and yaw (psi) angles
phi, theta, psi = symbols('phi, theta, psi')

# Define angular velocities
wx_inB, wy_inB, wz_inB = symbols('omega_x, omega_y, omega_z')

# Define roll, pitch, and yaw rotation matrices that describe the bus's orientation in the world frame
taux_inB, tauy_inB, tauz_inB, fz_inB = symbols('tau_x, tau_y, tau_z, f_z')

In [5]:
p_inW = Matrix([[px_inW],[py_inW], [pz_inW]])
v_inB = Matrix([[vx_inB],[vy_inB],[vz_inB]])
w_inB = Matrix([[wx_inB],[wy_inB], [wz_inB]])
Rx = Matrix([[1,0,0],[0, cos(phi), -sin(phi)],[0, sin(phi),  cos(phi)]])
Ry = Matrix([[ cos(theta), 0, sin(theta)],[0, 1,0], [-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0],[sin(psi),  cos(psi), 0],[0,0,1]])
R_ofB_inW = Rz @ Ry @ Rx
R_ofW_inB = R_ofB_inW.T

In [6]:
# Get the net torque vector
tau_inB = Matrix([[taux_inB],
                      [tauy_inB],
                      [tauz_inB]])
tau_inB = simplify(tau_inB)

# Get the gravity force and convert to body coordiantes
grav_inW = Matrix([[0.],
                       [0.],
                       [-mass * grav]])
grav_inB = R_ofW_inB @ grav_inW

# Get the net force vector
f_inB = grav_inB + Matrix([[0.],
                               [0.],
                               [fz_inB]])
f_inB = simplify(f_inB)

In [7]:
# Assemble the moment of inertia matrix
I_inB = Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])

In [8]:
v_inW = R_ofB_inW @ v_inB
xyz_dot = simplify(v_inW)
print("px, py, pz rate as a function of orientation and body-fixed velocities:")
N(xyz_dot,3)

px, py, pz rate as a function of orientation and body-fixed velocities:


⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s
⎢                                                                             
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c
⎢                                                                             
⎣                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ

in(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ))⎤
                             ⎥
os(ψ) - sin(ψ)⋅sin(θ)⋅cos(φ))⎥
                             ⎥
)                            ⎦

In [9]:
v_inB_dot = (1 / mass) * (f_inB - w_inB.cross(mass * v_inB))
v_inB_dot = simplify(v_inB_dot)
print("Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:")
N(v_inB_dot,3)

Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:


⎡       -ω_y⋅v_z + 1.0⋅ω_z⋅v_y + 9.81⋅sin(θ)       ⎤
⎢                                                  ⎥
⎢     1.0⋅ωₓ⋅v_z - ω_z⋅vₓ - 9.81⋅sin(φ)⋅cos(θ)     ⎥
⎢                                                  ⎥
⎣2.0⋅f_z - ωₓ⋅v_y + 1.0⋅ω_y⋅vₓ - 9.81⋅cos(φ)⋅cos(θ)⎦

In [10]:
# Define the transformation that takes roll, pitch, and yaw rates to body-fixed angular rates
ex = Matrix([[1], [0], [0]])
ey = Matrix([[0], [1], [0]])
ez = Matrix([[0], [0], [1]])
M = Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)

# Invert the transformation to get a matrix that takes body-fixed angular rates to 
# roll, pitch, and yaw rates
M_inv = simplify(M.inv())

# Convert the body-fixed angular rates to roll, pitch, and yaw rates
rpy_dot = simplify(M_inv@w_inB)
print("Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:")
N(rpy_dot,3)

Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:


⎡ωₓ + ω_y⋅sin(φ)⋅tan(θ) + ω_z⋅cos(φ)⋅tan(θ)⎤
⎢                                          ⎥
⎢         ω_y⋅cos(φ) - ω_z⋅sin(φ)          ⎥
⎢                                          ⎥
⎢         ω_y⋅sin(φ) + ω_z⋅cos(φ)          ⎥
⎢         ───────────────────────          ⎥
⎣                  cos(θ)                  ⎦

In [11]:
# Apply Euler's equation to get the time derivative of the angular velocities of the
# quadrotor in the quadrotor frame
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB))
w_inB_dot = simplify(w_inB_dot)
print("Time derivative of the angular velocities in the body-fixed frame:")
N(w_inB_dot, 3)

Time derivative of the angular velocities in the body-fixed frame:


⎡-0.739⋅ω_y⋅ω_z + 435.0⋅τₓ⎤
⎢                         ⎥
⎢0.739⋅ωₓ⋅ω_z + 435.0⋅τ_y ⎥
⎢                         ⎥
⎣        250.0⋅τ_z        ⎦

In [12]:
f = Matrix.vstack(xyz_dot,
                      v_inB_dot,
                      rpy_dot,
                      w_inB_dot)

In [13]:
N(f,3)

⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s
⎢                                                                             
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c
⎢                                                                             
⎢                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ
⎢                                                                             
⎢                                   -ω_y⋅v_z + 1.0⋅ω_z⋅v_y + 9.81⋅sin(θ)      
⎢                                                                             
⎢                                 1.0⋅ωₓ⋅v_z - ω_z⋅vₓ - 9.81⋅sin(φ)⋅cos(θ)    
⎢                                                                             
⎢                            2.0⋅f_z - ωₓ⋅v_y + 1.0⋅ω_y⋅vₓ - 9.81⋅cos(φ)⋅cos(θ
⎢                                                                             
⎢                                ωₓ + ω_y⋅sin(φ)⋅tan

In [14]:
# Position of markers in body frame
mark1_inB = Matrix([[lxy],
                        [0.],
                        [lz]])
mark2_inB = Matrix([[0.],
                        [lxy],
                        [lz]])
mark3_inB = Matrix([[-lxy],
                        [0.],
                        [lz]])
mark4_inB = Matrix([[0.],
                        [-lxy],
                        [lz]])

# Position of markers in world frame
mark1_inW = p_inW + R_ofB_inW @ mark1_inB
mark2_inW = p_inW + R_ofB_inW @ mark2_inB
mark3_inW = p_inW + R_ofB_inW @ mark3_inB
mark4_inW = p_inW + R_ofB_inW @ mark4_inB

# Sensor model
g = simplify(Matrix.vstack(mark1_inW, mark2_inW, mark3_inW, mark4_inW))

In [15]:
N(g,3)

⎡              pₓ + 0.0469⋅sin(φ)⋅sin(ψ) + 0.0469⋅sin(θ)⋅cos(φ)⋅cos(ψ) + 0.25⋅
⎢                                                                             
⎢              p_y - 0.0469⋅sin(φ)⋅cos(ψ) + 0.0469⋅sin(ψ)⋅sin(θ)⋅cos(φ) + 0.25
⎢                                                                             
⎢                                p_z - 0.25⋅sin(θ) + 0.0469⋅cos(φ)⋅cos(θ)     
⎢                                                                             
⎢pₓ + 0.0469⋅sin(φ)⋅sin(ψ) + 0.25⋅sin(φ)⋅sin(θ)⋅cos(ψ) - 0.25⋅sin(ψ)⋅cos(φ) + 
⎢                                                                             
⎢p_y + 0.25⋅sin(φ)⋅sin(ψ)⋅sin(θ) - 0.0469⋅sin(φ)⋅cos(ψ) + 0.0469⋅sin(ψ)⋅sin(θ)
⎢                                                                             
⎢                             p_z + 0.25⋅sin(φ)⋅cos(θ) + 0.0469⋅cos(φ)⋅cos(θ) 
⎢                                                                             
⎢              pₓ + 0.0469⋅sin(φ)⋅sin(ψ) + 0.0469⋅si

## System Linearization and State Space Form

In [16]:
# FIXME: Find an equilibrium point and linearize the system into state space form
px_e = py_e = pz_e = vx_e = vy_e = vz_e = phi_e = theta_e = psi_e = omegax_e = omegay_e = omegaz_e = taux_e = tauy_e = tauz_e = 0.
fz_e = 4.905

m_e = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
n_e = np.array([0, 0, 0, 4.905])

In [17]:
f_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], f)
f_eq = f_num(px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e, taux_e, tauy_e, tauz_e, fz_e)
Matrix(f_eq)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [18]:
# Solving for Coefficient Matrix A
A_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 f.jacobian([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB]))
A = A_num(px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e, taux_e, tauy_e, tauz_e, fz_e)
print(A.shape)
Matrix(A)

(12, 12)


⎡0  0  0  1.0   0    0     0     0    0   0    0    0 ⎤
⎢                                                     ⎥
⎢0  0  0   0   1.0   0     0     0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0   1.0    0     0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0    9.81  0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0   -9.81   0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0  1.0   0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0   0   1.0   0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0   0    0   1.0⎥
⎢                                               

In [19]:
# Solving for Coefficient Matrix B
B_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 f.jacobian([taux_inB, tauy_inB, tauz_inB, fz_inB]))
B = B_num(px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e, taux_e, tauy_e, tauz_e, fz_e)
print(B.shape)
Matrix(B)

(12, 4)


⎡       0                 0            0     0 ⎤
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0    2.0⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢434.782608695652         0            0     0 ⎥
⎢                                              ⎥
⎢       0          4

In [20]:
# Checking Controllability
n = A.shape[0]
Wc = B
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    Wc = np.block([Wc, col])
print(Wc.shape)
print(np.linalg.matrix_rank(Wc) == n)

(12, 48)
True


In [21]:
# Solving for equilibrium matrix g
g_eq = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], g)
g_eq = g_eq(px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e, taux_e, tauy_e, tauz_e, fz_e)
g_eq = g_eq.reshape(12,)
print(g_eq.shape)
Matrix(g_eq)

(12,)


⎡  0.25  ⎤
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢0.046875⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢  0.25  ⎥
⎢        ⎥
⎢0.046875⎥
⎢        ⎥
⎢ -0.25  ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢0.046875⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎢ -0.25  ⎥
⎢        ⎥
⎣0.046875⎦

In [22]:
# Solving for Coefficient Matrix C
C_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 g.jacobian([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB]))
C = C_num(px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e, taux_e, tauy_e, tauz_e, fz_e)
print(C.shape)
Matrix(C)

(12, 12)


⎡1.0   0    0   0  0  0      0      0.046875    0    0  0  0⎤
⎢                                                           ⎥
⎢ 0   1.0   0   0  0  0  -0.046875     0      0.25   0  0  0⎥
⎢                                                           ⎥
⎢ 0    0   1.0  0  0  0      0       -0.25      0    0  0  0⎥
⎢                                                           ⎥
⎢1.0   0    0   0  0  0      0      0.046875  -0.25  0  0  0⎥
⎢                                                           ⎥
⎢ 0   1.0   0   0  0  0  -0.046875     0        0    0  0  0⎥
⎢                                                           ⎥
⎢ 0    0   1.0  0  0  0    0.25        0        0    0  0  0⎥
⎢                                                           ⎥
⎢1.0   0    0   0  0  0      0      0.046875    0    0  0  0⎥
⎢                                                           ⎥
⎢ 0   1.0   0   0  0  0  -0.046875     0      -0.25  0  0  0⎥
⎢                                                           ⎥
⎢ 0    0

In [23]:
# Solving for Coefficient Matrix D
D_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 g.jacobian([taux_inB, tauy_inB, tauz_inB, fz_inB]))
D = D_num(px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e, taux_e, tauy_e, tauz_e, fz_e)
print(D.shape)
Matrix(D)

(12, 4)


⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

In [24]:
# Checking for Observability
n = np.shape(A)[0]
Wo = C
for i in range(1, n):
    col = C@np.linalg.matrix_power(A, i)
    Wo = np.block([[Wo], [col]])
print(Wo.shape)
print(np.linalg.matrix_rank(Wo) == n)

(144, 12)
True


In [25]:
def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

In [45]:
Qc = np.diag([1/(100**2), 1/(100**2), 1/(10**2), 1/(105**2), 1/(105**2), 1/(105**2), 1/(50**2), 1/(100**2), 1/(100**2), 1/(50**2), 1/(50**2), 1/(50**2)])
Rc = np.diag([1/(0.065**2),1/(0.065**2),1/(0.065**2),1/(16**2)])
K = lqr(A,B,Qc,Rc)
print(K.shape)
#Matrix(K)
print(K.tolist())

(4, 12)
[[2.488389416073022e-19, -0.0006499999999999957, -2.453366449180228e-17, 2.5052952214053916e-19, -0.0015437786460063903, -2.50120007078978e-17, 0.015092553343889915, 9.62709853181354e-19, -9.933972558517063e-19, 0.008433015201094638, -9.39808517467077e-19, -5.7844920497034e-18], [0.0006500000000000011, 2.762553015218238e-19, -4.854332352263061e-18, 0.0015400826569502335, 6.142992815446531e-19, -1.8198493649923982e-19, -3.478825393907785e-18, 0.015006542903954912, -1.5260510957650788e-19, -9.39808517467077e-19, 0.008409524205220694, 2.971017055523295e-18], [2.171392602961896e-19, 9.70809015260651e-19, 2.4665892861596243e-18, 4.479352252462763e-19, 1.2412654196256563e-18, -5.610210634061221e-19, -8.163456315495568e-18, 3.421997862882925e-18, 0.0006500000000000008, -3.326082928579456e-18, 1.7083348069258952e-18, 0.0026248809496813405], [7.358294879379037e-17, 9.11797240443686e-16, 1.59999999999999, 3.6953365809426163e-17, 1.279470439098354e-15, 1.2740564958621354, -1.5581274088751

In [46]:
Ro = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Qo = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
L = lqr(A.T,C.T,linalg.inv(Ro),linalg.inv(Qo)).T
print(L.shape)
#Matrix(L)
print(L.tolist())

(12, 12)
[[1.6029062077908953, 3.4241981533357485e-16, -0.19023458860096662, 1.6029062077908953, 3.3519870604282115e-16, -6.18544002171761e-17, 1.6029062077908953, 3.2797759675206746e-16, 0.19023458860096668, 1.6029062077908953, 3.3519870604282115e-16, 1.2208612393107435e-16], [3.4524039876789437e-16, 1.6029062077908918, -2.3396625344458764e-16, 2.1088812043332036e-16, 1.6029062077908915, -0.1902345886009658, 3.4524039876789437e-16, 1.6029062077908913, 5.708565977111054e-17, 4.795926771024684e-16, 1.6029062077908915, 0.19023458860096562], [2.985403526868438e-17, -2.1846278881175878e-16, 0.7071067811865477, 1.6265549442012165e-16, -8.56613296603215e-17, 0.7071067811865477, 2.985403526868438e-17, 4.714012949111578e-17, 0.7071067811865477, -1.029474238827529e-16, -8.56613296603215e-17, 0.7071067811865477], [4.910386974689184, 2.850457435659415e-15, -1.2564328215456562, 4.910386974689184, 2.6455790908358463e-15, -6.660698609814086e-16, 4.910386974689184, 2.4407007460122776e-15, 1.256432821

In [28]:
print(linalg.eigvals(A-B@K))
print(linalg.eigvals(A-L@C))

[-0.67722387+1.26659248j -0.67722387-1.26659248j -1.15093356+0.13888266j
 -1.15093356-0.13888266j -0.68837747+1.26361086j -0.68837747-1.26361086j
 -1.2740565 +1.25569903j -1.2740565 -1.25569903j -1.14488671+0.16787305j
 -1.14488671-0.16787305j -0.32811012+0.23418742j -0.32811012-0.23418742j]
[-1.40682801+2.17021669e+00j -1.40682801-2.17021669e+00j
 -1.40682801+2.17021669e+00j -1.40682801-2.17021669e+00j
 -2.93130819+0.00000000e+00j -2.93130819+0.00000000e+00j
 -0.55901699+4.33012702e-01j -0.55901699-4.33012702e-01j
 -1.00128528+0.00000000e+00j -1.00128528+0.00000000e+00j
 -1.41421356+4.78610025e-08j -1.41421356-4.78610025e-08j]


## Gain Selection and Controller Design

In [39]:
# Create the Controller class
class Controller():
    def __init__(self):
        # Initializing controller
        self.A = np.array(A, dtype=float)
        self.B = np.array(B, dtype=float)
        self.C = np.array(C, dtype=float)
        self.K = np.array(K, dtype=float)
        self.L = np.array(L, dtype=float)
        self.m_e = np.array([px_e, py_e, pz_e, vx_e, vy_e, vz_e, phi_e, theta_e, psi_e, omegax_e, omegay_e, omegaz_e])
        self.n_e = np.array([taux_e, tauy_e, tauz_e, fz_e])
        self.g_eq = g_eq
        self.e_max = 1.0
        self.phat_history = np.array([0.,0.,0.])
        self.pdes_history = np.array([0.,0.,0.])
        pass

    
    def reset(self):
        # Defining arrays for data collection
        self.xhat = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
        self.phat_history = np.array([0.,0.,0.])
        self.pdes_history = np.array([0.,0.,0.])
        pass
    
    
    def run(self, **kwargs):
        # Obtaining necessary variables
        self.dt = kwargs["dt"]
        self.p_des = kwargs["next_gate"]
        self.p_dir = kwargs["dir_gate"]
        self.p_hat = self.xhat[:3]

        # Defining targeted position of gate 
        up = np.array([0,0,1])
        self.p_des = self.p_des + up
        
        # Constants
        k_attract = 11.9    # ideal is 11.5
        k_repel = 0        # ideal is 1.0
        k_des = 1.0        # ideal is 1.0
        r_drone = 0.8      # ideal is 0.8
        
        # Attractive Component
        h_attract = k_attract*(self.p_hat - self.p_des)/np.linalg.norm(self.p_hat - self.p_des)

        # Repulsive Component
        h_repel = 0
        for q in kwargs["pos_others"]:
            p_obst = q + r_drone*(self.p_hat - q)/np.linalg.norm(self.p_hat - q)
            grad_d = (self.p_hat - p_obst)/np.linalg.norm(self.p_hat - p_obst)
            d_p = np.linalg.norm(self.p_hat - p_obst) - r_drone
            h_repel += -k_repel*(grad_d/d_p**2)
        
        h = h_attract + h_repel
        
        # Minimizing error
        if np.linalg.norm(self.p_des - self.p_hat) < self.e_max:
            self.p_des = self.p_des
        else:
            self.p_des = self.p_hat - k_des*h
            
        self.x_des = np.block([self.p_des, np.zeros(9)])

        # Calculating gains
        u = - self.K@(self.xhat - self.x_des)
        inputs = u + self.n_e
            
        # Last gate condition
        if kwargs["is_last_gate"]:
            inputs = inputs*1.1
        
        # Updating state estimate
        sensor = np.block([kwargs["mocap_1"], kwargs["mocap_2"], kwargs["mocap_3"], kwargs["mocap_4"]])
        y = sensor - self.g_eq
        self.xhat += self.dt * (self.A@self.xhat + self.B@u - self.L@(self.C@self.xhat - y))

        # Updating observer history
        self.phat_history = np.block([[self.phat_history], [self.xhat[:3]]])
        self.pdes_history = np.block([[self.pdes_history], [self.p_des]])
        
        return inputs

# Creates an instance of the Controller class to simulate a quadrotor
controller = Controller()

In [43]:
# Creating empty data arrays
p_est_act = np.array([])
p_des_act = np.array([])
time = np.array([])

trials = 1

# Initialize the controller
controller.__init__()
for i in range(1, trials+1):
    # Reset the simulator with initial conditions at equilibirum
    team_list = ["Team Flying Mambas"]
    sim = ae353_quadrotor.Quadrotor_Sim(n_quadrotors=1,
                                    team_list=team_list,
                                    planet="Earth",
                                    use_keyboard=False,
                                    visualization=False,
                                    visualization_fr=45.,
                                    animation=False,
                                    verbose=False)
    # Reset the controller
    controller.reset()
    
    # Run the simulation
    controllers = [controller]
    (track_stats, data) = sim.run(controllers,
                              sensor_noise=0.01,
                              collect_data=True,
                              max_time=25.0,
                              verbose=True)
    # Collecting data
    p_est = controller.phat_history
    p_des = controller.pdes_history
    pos = np.array(data[0]['position'])

    # Storing data into arrays
    p_est_act = np.append(p_est_act, np.sqrt(np.mean(pos - p_est[1:])**2))
    p_des_act = np.append(p_des_act, np.sqrt(np.mean(pos - p_des[1:])**2))

    # Collecting simulation run times
    if(data[0]["time"][-1] < 29):
        time = np.append(time, data[0]["time"][-1])
    else:
        time = np.append(time, 0)

Team Flying Mambas:0 PASSED GATE 1 IN 3.33s
Team Flying Mambas:0 PASSED GATE 2 IN 6.12s
Team Flying Mambas:0 PASSED GATE 3 IN 8.74s
Team Flying Mambas:0 PASSED GATE 4 IN 12.24s
Team Flying Mambas:0 PASSED GATE 5 IN 15.65s
Team Flying Mambas:0 PASSED GATE 6 IN 21.53s
Team Flying Mambas:0 PASSED GATE 7 IN 23.21s
Team Flying Mambas:0 COMPLETED LAP IN 23.21s
